## Importando as bibliotecas

- tensorflow: biublioteca para processamento de dados e contrução de CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

## Definindo a localização dos dados

Armazena o local onde os dados de treino e validação estão presentes

In [ ]:
TRAIN_DIR = "../data/proof_of_concept/train"
VAL_DIR = "../data/proof_of_concept/val"

## Carregamento do dataset

- image_size: muda as imagens para o tamanho de 224X224 pixels
- batch_size: define o número de imagens por lote
- label_mode: separa todas as imagens em duas classes

In [9]:
train_data = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, image_size=(224, 224), batch_size=32, label_mode="binary"
)

val_data = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR, image_size=(224, 224), batch_size=32, label_mode="binary"
)

Found 7995 files belonging to 2 classes.
Found 10 files belonging to 2 classes.


##  Pré processamento das imagens

#### Noramalização

- normalization_layer: deffine para que os pixels da imagem sejam normalizados para valores entre 0 e 1

- Realiza a normalização nos conjuntos de treino e de validação

In [10]:
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))

#### Denfinindo e aplicando padrões de aumentação

- RandomFlip: inverte a imagem na horizontal
- RandomRotation: gira a imagem
- RandomZoom: dá zoom na imagem

In [11]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.05),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

### Data augmentation

Realiza o data augmentation apenas no conjunto de teste

In [16]:
train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

## Otimização do desempenho

O AUTOTUNE permite que a biblioteca do TensorFloe decida sozinho quantos dados carregar com antecedência, quando carregar e quanto de paralelismo usar, para se ajustar automaticamente para o melhor desempenho no hardware disponível

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.prefetch(buffer_size=AUTOTUNE)
val_data = val_data.prefetch(buffer_size=AUTOTUNE)

## Carregamento do Modelo Pré-treinado (transfer learning)

- Carrega o modelo base ResNet50 pré-treinada no Imagenet

- Congela as camadas da rede base

In [27]:
base_model = tf.keras.applications.ResNet50(
  weights="imagenet", include_top=False, input_shape=(224, 224, 3)
)

base_model.trainable = False

## Construção do modelo final

Diz ao modelo que ele receberá imagens de 224X224 pixels com 3 canais de cor

In [70]:
inputs = tf.keras.Input(shape=(224,224,3))


Usado para impedir instabilidades no treinamento ou destruição do conhecimento prévio do modelo pré-treinado

- x representa os dados intermediários do processamento

In [71]:
x = base_model(inputs, training =False)

Reduz a dimensionabilidade das características extraídas  pelo ResNet
- A rede neural desenha vários mapas das características da imagem
- O GlobalAveragePooling2D tira a média de cada um destes mapas
- Reduz o número de parâmetros, melhora a generalização e diminui overfitting

In [72]:
x = GlobalAveragePooling2D()(x)

É uma camada de decisão
- Combina as características resumidas pelo GlobalAveragePooling2D, aprende as relações
- 128 indica o número de neurônios pelos quais as informações pasarão
  - números menores fazem o modelo decidir muito rápido, sem "raciocinar" direito
  - números maiores podem tornar o modelo pesado demais, além de poderem fazer ele "se confundir"
- O ReLU (Rectified Linear Unit) age como um filtro de relevância
  - valores negativos são transformados em zero - informações
  - valores ppositivos são mantidos

In [73]:
x = Dense(128, activation='relu')(x)

Outra camada de decisão
- O valor 1 indica que deve haver apenas uma saída
- O sigmoid faz o modelo decidir entre uma resposta binária
  - 0.34 vira 0
  - 0.73 vira 1

In [74]:
outputs = Dense(1,activation='sigmoid')(x)

Informa ao modelo quais tipos de dados serão sua entrada e o tipo de saída esperada após o processamento
- o modelo automaticamente conecta todas as epatas interemediárias

In [75]:
model = Model(inputs, outputs)

## Compilação do Modelo

- Optimezer indica ao modelo como ajustar seus erros
  - Adam é um otimizador rápido e estável, bom para a maioria dos problemas
- Loss é uma forma de medir o quão longe o modelo estava da resosta certa
  - binary_crossebtropy é usada quando há duas classes e a saída é uma probabilidade entre 0 e 1
  - Pune mais erros confiantes que erros "em dúvida"
- Metrics ajuda a acompanhar o desempenho do modelo, se ele está realmente aprendendo
  - Accuray é basicamente a precisão do modelo, quantas previsões ele acertou


In [77]:
model.compile(
  optimizer="adam",
  loss="binary_crossentropy",
  metrics =["accuracy"]
)

## Treinamento do modelo

Indica ao modelo o conjunto de treinamento, de validação e o núemero de épocas

- Épocas indica o número de vezes que o modelo vê todo o conjunto de dados

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=10)